In [2]:
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd


recrutamento = pd.read_csv('recruitment_data.csv', sep=',', decimal='.')

features = list(recrutamento.columns.values)
classificacao = 'HiringDecision'

features.remove(classificacao)

X = recrutamento[features].to_numpy() 
Y = recrutamento[classificacao].to_numpy() 

import pandas as pd
from sklearn.model_selection import train_test_split

X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X,
                                                                  Y,
                                                                  test_size = 0.3,
                                                                  random_state = 22)



from tensorflow.keras.models import Sequential

autoencoder = Sequential([
    layers.InputLayer((X_treinamento.shape[1],)),
    layers.Dense(5, activation='relu'),
    layers.Dense(1, activation='relu', name='bottleneck'),
    layers.Dense(5, activation='relu'),
    layers.Dense(X_treinamento.shape[1], activation='sigmoid')
])

autoencoder.compile(optimizer='adam', loss='mse')

# Treinar o autoencoder
autoencoder.fit(X_treinamento, X_treinamento, epochs=500, batch_size=32, validation_data=(X_teste, X_teste), verbose=0)

# Extrair as características do codificador
encoder = Sequential(autoencoder.layers[:2])
X_train_encoded = encoder.predict(X_treinamento)
X_test_encoded = encoder.predict(X_teste)



In [7]:
import matplotlib.pyplot as plt
#IMPLEMENTANDO O RANDOMFOREST SEM BALANCEAMENTO
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay


floresta = RandomForestClassifier(n_estimators = 100)
floresta.fit(X_train_encoded, y_treinamento)

previsoes = floresta.predict(X_test_encoded)

# Os valores percentuais da previsão
# predict_proba retorna a probabilidade de cada classe, o primeiro valor é a probabilidade de ser 0 e o segundo de ser 1
previsoes_percent = floresta.predict_proba(X_test_encoded)
# As probabilidades de 1, usaremos para a função ROC
previsoes_percent = previsoes_percent[:,1]
previsoes_percent


matriz_confusao = confusion_matrix(y_teste, previsoes)
matriz_confusao

# Acurácia
accuracy = accuracy_score(y_teste, previsoes)
print("Acurácia:", accuracy)

# Precisão
precision = precision_score(y_teste, previsoes)
print("Precisão:", precision)

# Recall
recall = recall_score(y_teste, previsoes)
print("Recall:", recall)

# Especificidade.
tn, fp, fn, tp = confusion_matrix(y_teste, previsoes).ravel()
specificity = tn / (tn + fp)
print("Especificidade:", specificity)

# F1 Score
f1 = f1_score(y_teste, previsoes)
print("F1 Score:", f1)

# ROC AUC Score (necessita das probabilidades de classe em vez de previsões).
# Para calcular a ROC AUC é necessário passar a probabilidade de ser 1.
roc_auc = roc_auc_score(y_teste, previsoes_percent)
print("ROC AUC Score:", roc_auc)

fig = ConfusionMatrixDisplay.from_estimator(floresta, X_test_encoded, y_teste, display_labels=['Not Hired', 'Hired'], cmap='Greens')
plt.title('Standard Random Forest Confusion Matrix')
plt.show()



Acurácia: 0.60
